In [1]:
import pandas as pd


In [2]:
train = pd.read_csv('../DataSet/train.csv')
test = pd.read_csv('../DataSet/test.csv')

In [3]:
train.dtypes

month_07              int64
month_08              int64
month_09              int64
Temperature         float64
RH                  float64
Rain                float64
FFMC                float64
log_skewed_FFMC     float64
Classes_not fire      int64
dtype: object

In [4]:
train['month_07'] = train['month_07'].astype('uint8')
train['month_08'] = train['month_08'].astype('uint8')
train['month_09'] = train['month_09'].astype('uint8')
train['Classes_not fire'] = train['Classes_not fire'].astype('uint8')

test['month_07'] = test['month_07'].astype('uint8')
test['month_08'] = test['month_08'].astype('uint8')
test['month_09'] = test['month_09'].astype('uint8')
test['Classes_not fire'] = test['Classes_not fire'].astype('uint8')

In [5]:
train.dtypes

month_07              uint8
month_08              uint8
month_09              uint8
Temperature         float64
RH                  float64
Rain                float64
FFMC                float64
log_skewed_FFMC     float64
Classes_not fire      uint8
dtype: object

In [6]:
test.dtypes

month_07              uint8
month_08              uint8
month_09              uint8
Temperature         float64
RH                  float64
Rain                float64
FFMC                float64
log_skewed_FFMC     float64
Classes_not fire      uint8
dtype: object

In [7]:
train.head()

,month_07,month_08,month_09,Temperature,RH,Rain,FFMC,log_skewed_FFMC,Classes_not fire
0,0,0,0,0.266381,-0.821244,-0.381541,0.774336,1.291430,0
1,1,0,0,0.539809,1.254823,-0.381541,0.300430,3.328559,1
2,0,0,1,-0.827331,0.183304,-0.104489,-0.626776,0.000000,1
3,1,0,0,0.539809,-1.156094,-0.381541,0.897963,1.113632,0
4,0,0,1,-0.553903,1.187853,-0.381541,0.389717,2.565966,0


In [8]:
test.head()

,month_07,month_08,month_09,Temperature,RH,Rain,FFMC,log_skewed_FFMC,Classes_not fire
0,0,1,0,0.539809,0.183304,-0.381541,0.643840,1.553181,0
1,1,0,0,-0.280475,1.120883,-0.381541,0.547685,1.825866,0
2,0,1,0,1.906949,-1.557913,-0.289191,0.815545,1.226174,0
3,0,0,1,-1.100759,1.455733,-0.381541,0.437794,2.284178,0
4,0,0,1,-0.280475,-0.486395,-0.381541,0.712522,1.403466,0


In [9]:
X_train = train.iloc[:,:-1]
X_test = test.iloc[:,:-1]

#X_test.head()
X_train.head()

,month_07,month_08,month_09,Temperature,RH,Rain,FFMC,log_skewed_FFMC
0,0,0,0,0.266381,-0.821244,-0.381541,0.774336,1.291430
1,1,0,0,0.539809,1.254823,-0.381541,0.300430,3.328559
2,0,0,1,-0.827331,0.183304,-0.104489,-0.626776,0.000000
3,1,0,0,0.539809,-1.156094,-0.381541,0.897963,1.113632
4,0,0,1,-0.553903,1.187853,-0.381541,0.389717,2.565966


In [10]:
y_train = train['Classes_not fire']
y_test = test[['Classes_not fire']]
y_train.head()

0    0
1    1
2    1
3    0
4    0
Name: Classes_not fire, dtype: uint8

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [12]:
lr = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
adaBoost = AdaBoostClassifier()
lgbm = LGBMClassifier()
gbt = GradientBoostingClassifier()

In [17]:
clfs = {
    'LogisticRegression': lr,
    'SVC': svc,
    'KNeighbors': knn,
    'RandomForest': rf,
    'AdaBoost': adaBoost,
    
    'GradientBoostingTrees': gbt
}

In [24]:
param_grid = {
    
    'LogisticRegression':{'penalty':['elasticnet','l3'],
                          'solver':['saga','lbfgs'],
                          'max_iter':[1500,2000],
                          'l1_ratio':[0.5,0.4,0.2,0,0.7,0.8,0.9,1],                          
                          'C': [10e-3,10e-2,10e-1,1.0],
                         'class_weight': ['balanced',None]},
    'SVC': {'C':[10e-4,10e-3,10e-2,10e-1,1.0],
           'kernel':['linear','rbf','sigmoid'],
           'gamma':['scale','auto']},
    
    'KNeighbors': {'n_neighbors':[3,4,5,7,9,10,11,13],  # n_neighbors should always be odd numbers
                     'weights': ['uniform','distance']},
    
    'RandomForest': {'n_estimators':[100,200,300],
                    'criterion':['gini','log_loss'],
                    'max_depth':[2,3,4,5,6,7,8,9,10,12,14,15,18],
                    'class_weight': ['balanced',None]},
    
    'AdaBoost': {'n_estimators': [10,20,30,40,50,60,70,100],
                'learning_rate': [10e-3,10e-2,10e-1,1.0]},
    
    'GradientBoostingTrees': {'loss': ['log_loss', 'exponential'],
                             'learning_rate': [10e-3,10e-2,10e-1,1.0],
                             'n_estimators': [100,200,250],
                             'max_depth': [2,3,4,5,6,7,8,9,10]},
    
           
    }

In [25]:
def train_classifier(clf,parameters):
    
    gs = GridSearchCV(clf,param_grid=parameters, cv=5,refit='f1', verbose=3, scoring=['f1','roc_auc']).fit(X_train, y_train)
   
    #y_pred = gs.predict(X_test)    
    
    return gs

In [26]:
tuned_models = []

for name,clf in clfs.items():
    
    print("\nFor ",name)
    parameters = param_grid[name]
    current_best_model = train_classifier(clf,parameters)
    
    tuned_models.append(current_best_model)


For  LogisticRegression
Fitting 5 folds for each of 512 candidates, totalling 2560 fits
[CV 1/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.903) roc_auc: (test=0.995) total time=   0.0s
[CV 2/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.800) roc_auc: (test=0.997) total time=   0.0s
[CV 3/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.759) roc_auc: (test=0.995) total time=   0.0s
[CV 4/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.941) roc_auc: (test=1.000) total time=   0.0s
[CV 5/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=1.000) roc_auc: (test=1.000) total time=   0.0s
[CV 1/5] END C=0.01, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet

C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.971) roc_auc: (test=1.000) total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.909) roc_auc: (test=0.989) total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.909) roc_auc: (test=0.976) total time=   0.0s


C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=saga; f1: (test=0.971) roc_auc: (test=0.997) total time=   0.0s
[CV 1/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 2/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 3/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 4/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 5/5] END C=0.1, class_weight=balanced, l1_ratio=0.5, max_iter=1500, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 1/5] END C=0.1, class_weight=bala

...

[CV 2/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=saga; f1: (test=0.971) roc_auc: (test=1.000) total time=   0.0s
[CV 3/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=saga; f1: (test=0.919) roc_auc: (test=0.992) total time=   0.0s
[CV 4/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=saga; f1: (test=0.941) roc_auc: (test=0.976) total time=   0.0s


C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1920 fits failed out of a total of 2560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
640 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\HIM

[CV 5/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=saga; f1: (test=0.971) roc_auc: (test=0.992) total time=   0.0s
[CV 1/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 2/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 3/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 4/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 5/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty=elasticnet, solver=lbfgs; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 1/5] END C=1.0, class_weight=None, l1_ratio=1, max_iter=2000, penalty

C:\Users\HIMANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END C=0.001, gamma=scale, kernel=rbf; f1: (test=0.000) roc_auc: (test=1.000) total time=   0.0s
[CV 3/5] END C=0.001, gamma=scale, kernel=rbf; f1: (test=0.000) roc_auc: (test=0.974) total time=   0.0s
[CV 4/5] END C=0.001, gamma=scale, kernel=rbf; f1: (test=0.000) roc_auc: (test=0.989) total time=   0.0s
[CV 5/5] END C=0.001, gamma=scale, kernel=rbf; f1: (test=0.000) roc_auc: (test=0.994) total time=   0.0s
[CV 1/5] END C=0.001, gamma=scale, kernel=sigmoid; f1: (test=0.000) roc_auc: (test=0.735) total time=   0.0s
[CV 2/5] END C=0.001, gamma=scale, kernel=sigmoid; f1: (test=0.000) roc_auc: (test=0.532) total time=   0.0s
[CV 3/5] END C=0.001, gamma=scale, kernel=sigmoid; f1: (test=0.000) roc_auc: (test=0.688) total time=   0.0s
[CV 4/5] END C=0.001, gamma=scale, kernel=sigmoid; f1: (test=0.000) roc_auc: (test=0.701) total time=   0.0s
[CV 5/5] END C=0.001, gamma=scale, kernel=sigmoid; f1: (test=0.000) roc_auc: (test=0.613) total time=   0.0s
[CV 1/5] END C=0.001, gamma=auto, k

In [27]:
tuned_models

[GridSearchCV(cv=5, estimator=LogisticRegression(),
              param_grid={'C': [0.01, 0.1, 1.0, 1.0],
                          'class_weight': ['balanced', None],
                          'l1_ratio': [0.5, 0.4, 0.2, 0, 0.7, 0.8, 0.9, 1],
                          'max_iter': [1500, 2000],
                          'penalty': ['elasticnet', 'l3'],
                          'solver': ['saga', 'lbfgs']},
              refit='f1', scoring=['f1', 'roc_auc'], verbose=3),
 GridSearchCV(cv=5, estimator=SVC(),
              param_grid={'C': [0.001, 0.01, 0.1, 1.0, 1.0],
                          'gamma': ['scale', 'auto'],
                          'kernel': ['linear', 'rbf', 'sigmoid']},
              refit='f1', scoring=['f1', 'roc_auc'], verbose=3),
 GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
              param_grid={'n_neighbors': [3, 4, 5, 7, 9, 10, 11, 13],
                          'weights': ['uniform', 'distance']},
              refit='f1', scoring=['f1', 'roc_auc'], 

In [50]:
import os
import pickle
import re

# create the directory if it does not exist
if not os.path.exists('../tuned_models'):
    os.makedirs('../tuned_models')

for model in tuned_models:
    filename = 'tuned_' + re.search(r'^[^\(]+', str(model.best_estimator_))[0] + '_model.pkl'
    filepath = os.path.join('../tuned_models', filename)
    with open(filepath, 'wb') as file:
        pickle.dump(model, file)


In [60]:
SVM_model = pickle.load(open('../tuned_models/tuned_SVC_model.pkl','rb'))
AdaBoost_model = pickle.load(open('../tuned_models/tuned_AdaBoostClassifier_model.pkl','rb'))
GBDT_model = pickle.load(open('../tuned_models/tuned_GradientBoostingClassifier_model.pkl','rb'))
knn_model = pickle.load(open('../tuned_models/tuned_KNeighborsClassifier_model.pkl','rb'))
lr_model = pickle.load(open('../tuned_models/tuned_LogisticRegression_model.pkl','rb'))
RF_model = pickle.load(open('../tuned_models/tuned_RandomForestClassifier_model.pkl','rb'))

In [63]:
model_scores = pd.DataFrame({
    'model': ['SVM','KNN','AdaBoost','GBDT','RF','LR'],
    'balanced_accuracy_train': [balanced_accuracy_score(y_train,svm_model.predict(X_train)),
                                balanced_accuracy_score(y_train,knn_model.predict(X_train)),
                                balanced_accuracy_score(y_train,AdaBoost_model.predict(X_train)),
                                balanced_accuracy_score(y_train,GBDT_model.predict(X_train)),
                                balanced_accuracy_score(y_train,RF_model.predict(X_train)),
                                balanced_accuracy_score(y_train,lr_model.predict(X_train)),
                               ]})


balanced_accuracy_test =  [balanced_accuracy_score(y_test,svm_model.predict(X_test)),
                           balanced_accuracy_score(y_test,knn_model.predict(X_test)),
                           balanced_accuracy_score(y_test,AdaBoost_model.predict(X_test)),
                           balanced_accuracy_score(y_test,GBDT_model.predict(X_test)),
                           balanced_accuracy_score(y_test,RF_model.predict(X_test)),
                           balanced_accuracy_score(y_test,lr_model.predict(X_test)),                           
                          ] 

f1_score_train = [f1_score(y_train,svm_model.predict(X_train)),
                  
                  f1_score(y_train,knn_model.predict(X_train)),
                  f1_score(y_train,AdaBoost_model.predict(X_train)),
                  f1_score(y_train,GBDT_model.predict(X_train)),
                  f1_score(y_train,RF_model.predict(X_train)),
                  f1_score(y_train,lr_model.predict(X_train)),
                 ]


f1_score_test = [f1_score(y_test,svm_model.predict(X_test)),
                 
                 f1_score(y_test,knn_model.predict(X_test)),
                 f1_score(y_test,AdaBoost_model.predict(X_test)),
                 f1_score(y_test,GBDT_model.predict(X_test)),
                 f1_score(y_test,RF_model.predict(X_test)),
                 f1_score(y_test,lr_model.predict(X_test)),
                ]

roc_auc_train =  [roc_auc_score(y_train,svm_model.predict(X_train)),
                  
                  roc_auc_score(y_train,knn_model.predict(X_train)),
                  roc_auc_score(y_train,AdaBoost_model.predict(X_train)),
                  roc_auc_score(y_train,GBDT_model.predict(X_train)),
                  roc_auc_score(y_train,RF_model.predict(X_train)),
                  roc_auc_score(y_train,lr_model.predict(X_train)),
                 ]

roc_auc_test = [roc_auc_score(y_test,svm_model.predict(X_test)),
                
                roc_auc_score(y_test,knn_model.predict(X_test)),
                roc_auc_score(y_test,AdaBoost_model.predict(X_test)),
                roc_auc_score(y_test,GBDT_model.predict(X_test)),
                roc_auc_score(y_test,RF_model.predict(X_test)),
                roc_auc_score(y_test,lr_model.predict(X_test)),
                ] 

precision_train =  [precision_score(y_train,svm_model.predict(X_train)),
                    
                    precision_score(y_train,knn_model.predict(X_train)),
                    precision_score(y_train,AdaBoost_model.predict(X_train)),
                    precision_score(y_train,GBDT_model.predict(X_train)),
                    precision_score(y_train,RF_model.predict(X_train)),
                    precision_score(y_train,lr_model.predict(X_train)),
                   ]

precision_test = [precision_score(y_test,svm_model.predict(X_test)),
                  
                  precision_score(y_test,knn_model.predict(X_test)),
                  precision_score(y_test,AdaBoost_model.predict(X_test)),
                  precision_score(y_test,GBDT_model.predict(X_test)),
                  precision_score(y_test,RF_model.predict(X_test)),
                  precision_score(y_test,lr_model.predict(X_test)),
                 ] 


accuracy_train =  [accuracy_score(y_train,svm_model.predict(X_train)),
                   
                   accuracy_score(y_train,knn_model.predict(X_train)),
                   accuracy_score(y_train,AdaBoost_model.predict(X_train)),
                   accuracy_score(y_train,GBDT_model.predict(X_train)),
                   accuracy_score(y_train,RF_model.predict(X_train)),
                   accuracy_score(y_train,lr_model.predict(X_train)),
                 ]


accuracy_test =  [accuracy_score(y_test,svm_model.predict(X_test)),
                  accuracy_score(y_test,knn_model.predict(X_test)),
                  accuracy_score(y_test,AdaBoost_model.predict(X_test)),
                  accuracy_score(y_test,GBDT_model.predict(X_test)),
                  accuracy_score(y_test,RF_model.predict(X_test)),
                  accuracy_score(y_test,lr_model.predict(X_test)),
                 ] 





best_score= [svm_model.best_score_, knn_model.best_score_, AdaBoost_model.best_score_,
          GBDT_model.best_score_, RF_model.best_score_, lr_model.best_score_]

#################################################################################

model_scores['balanced_accuracy_test'],model_scores['f1_train'],model_scores['f1_test'],model_scores['roc_auc_train'],model_scores['roc_auc_test'],model_scores['precision_train'],model_scores['precision_test'],model_scores['accuracy_train'],model_scores['accuracy_test'] = [balanced_accuracy_test,f1_score_train,f1_score_test,roc_auc_train,roc_auc_test, precision_train,precision_test,accuracy_train,accuracy_test]

model_scores.insert(1,'best_score',best_score)

##############################################################################
model_scores.to_csv('../tuned_models/model_scores.csv')
########################

model_scores

,model,best_score,balanced_accuracy_train,balanced_accuracy_test,f1_train,f1_test,roc_auc_train,roc_auc_test,precision_train,precision_test,accuracy_train,accuracy_test
0,SVM,0.965863,0.969768,0.967742,0.966292,0.947368,0.969768,0.967742,0.955556,0.900000,0.969072,0.959184
1,KNN,0.963820,0.972556,0.983871,0.971098,0.972973,0.972556,0.983871,0.988235,0.947368,0.974227,0.979592
2,AdaBoost,0.959593,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,GBDT,0.959593,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
4,RF,0.964405,0.968804,0.967742,0.965909,0.947368,0.968804,0.967742,0.965909,0.900000,0.969072,0.959184
5,LR,0.938326,0.944147,0.983871,0.940476,0.972973,0.944147,0.983871,0.987500,0.947368,0.948454,0.979592


# Learning Curve

In a classification problem, the learning curve shows the performance of a model on the training set and the validation set over increasing numbers of training samples. It shows how the performance of the model changes as the training set size increases. The performance metric used for classification learning curves is usually accuracy or F1 score.

In a regression problem, the learning curve shows the performance of a model on the training set and the validation set over increasing numbers of training samples. It shows how the performance of the model changes as the training set size increases. The performance metric used for regression learning curves is usually mean squared error (MSE) or R-squared.

In [134]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve



   
# split dataset into training and test data
train_sizes, train_scores, test_scores = learning_curve(GBDT_model, X_train, y_train, cv=2, train_sizes=np.linspace(0.1, 1.0, 5))




# calculate mean and standard deviation of training and test scores
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# plot learning curve
plt.figure()
plt.title('Learning Curve')
plt.xlabel('Training examples')
plt.ylabel('Score')
plt.grid()
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color='g')
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score')
plt.legend(loc='best')
plt.show()



Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV 1/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=100; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 2/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=100; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 3/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=100; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 4/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=100; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 5/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=100; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 1/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=200; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 2/5] END learning_rate=0.01, loss=log_loss, max_depth=2, n_estimators=200; f1: (test=nan) roc_auc: (test=nan) total time=   0.0s
[CV 3

KeyboardInterrupt: 